# PERSONAS - "The Club Benefits Programme" FIFA

# Construcción del conjunto de datos

Para codificar, comience importando las siguientes bibliotecas:

In [ ]:
#Instalamos Faker debido a que no viene instalado por defecto en Colab
!pip install Faker
#Importamos pandas para manejo de dataframes
import pandas as pd
#Importamos uuid para que solo exista 1 ID
import uuid
#Importamos random para la asignacion de datos randómicos
import random
#Importamos Faker para hacer uso de su recreacion de datos
from faker import Faker
#Importamos fecha para el manejo de fechas
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


**Size**

El tamaño del conjunto de datos será de 5000 puntos de datos. Asigné esta cantidad a una variable constante, que usé en todo momento:

In [ ]:
num_directives = 20000

**Características**

Elegí 4 características para cada persona. Estas características y los respectivos tipos de datos son:

     - ID: una cadena única de caracteres para identificar a cada persona.
     - Genero (Sexo:) tipo de datos de cadena de tres opciones.
     - NombreDir (Nombre:) tipo de datos de cadena del nombre y apellido del persona.
     - Correo (Correo electrónico:) tipo de datos de cadena de la dirección de correo electrónico del persona.

Ingresé lo anterior como una lista de características para inicializar un marco de datos de Pandas:

In [ ]:
#Lista de 7 atributos/features para los directivos
features = [
    #ID: una cadena única de caracteres para identificar a cada persona.
    "IdP",
    #Name (Nombre:) tipo de datos de cadena del nombre y apellido de la persona.
    "NombreP",
    #Gender (Sexo:) tipo de datos de cadena de tres opciones. Masculino, Femenino y No binario
    "GeneroP",
    #Email (Correo electrónico:) tipo de datos de cadena de la dirección de correo electrónico de la persona.
    "CorreoP"
]
#Generemos el dataframe donde estarán presentes
df = pd.DataFrame(columns=features)

**Creación de datos**

**IDs**

Para el atributo ID, utilicé la biblioteca uuid para generar una cadena aleatoria de caracteres 100 000 veces. Luego, lo asigné al atributo ID en el marco de datos.

In [ ]:
# Definimos en ID la función uuid para el análisis de cada uno de los 5000 datos
df['IdP'] = [uuid.uuid4().hex for i in range(num_directives)]

**UUID es una gran biblioteca para generar identificaciones únicas para cada usuario debido a su posibilidad astronómicamente baja de duplicar una identificación.** Es una gran opción cuando se trata de generar conjuntos de caracteres de identificación únicos. Pero, si desea asegurarse de que no se repitieron las ID, puede realizar una verificación simple en el marco de datos con lo siguiente:

In [ ]:
print(df['IdP'].nunique()==num_directives)

True


Esto devolverá True si todas las ID en el conjunto de datos son únicas.

**Gender**

**Este atributo es uno de los casos en los que probablemente no se debería utilizar una elección igualmente aleatoria. Porque se puede suponer con seguridad que cada elección no tiene la misma probabilidad de ocurrir.**

Para el género, proporcioné tres opciones: masculino, femenino y No binario.

In [ ]:
genders = ["M", "F", "NB"]

In [ ]:
df['GeneroP'] = random.choices(
    genders, 
    weights=(47,47,6), 
    k=num_directives
)

**Name**

Aquí usé la biblioteca Faker para crear miles de nombres para todos estos usuarios. **La biblioteca Faker es excelente en esta situación porque tiene una opción para nombres masculinos y femeninos.**}} Para procesar los nombres de género, creé una función para asignar nombres en función de un género determinado.

In [ ]:
# Instantiating faker
faker = Faker()

def name_gen(gender):
    """
    Parametros
    ----------
    genero
        Recibe el genero 
    Procedimiento
    -------------
        Genera un nombre en base a si el dato es masculino o femenino
    Retorna
    -------
        El nombre asignado
    """
    #Condicional para determinar si es hombre o mujer
    if gender=='M':
        return faker.name_male()
    elif gender=='F':
        return faker.name_female()
    
    return faker.name()# Genera los nombres para cada persona
# Guarda los datos en el data frame en base a los generos asignados
df['NombreP'] = [name_gen(i) for i in df['GeneroP']]

Usé mi función simple para producir rápidamente una lista de nombres basada en los datos del atributo "Género" antes y la asigné al marco de datos.

**Email**

Se creo una nueva función que daría formato a los nombres en direcciones de correo electrónico con un nombre de dominio predeterminado. La asignación del dominio sería exclusivo de la FIFA. También manejaría direcciones duplicadas simplemente agregando un número aleatorio al final del nombre formateado:

In [ ]:
def emailGen(name, duplicateFound=False):
    """
    Parametros
    ----------
    name
        Recibe el nombre del dato
    duplicateFound
        Analisa si hay duplicado 
    Procedimiento
    -------------
        Genera un correo aleatorio segun el nombre recibido
        Añade numeros en caso de encontrar duplicados
    Retorna
    -------
        Direccion del correo
    """
    # Dominio falso a usar
    dom = "@fifa.org"
    
    # Poner en minisculas el nombre recibido
    name = name.lower().split(" ")
    
    # Caracteres cualquiera para añadir al nombre
    chars = [".", "_"]
    
    new_name = name[0] + random.choice(chars) + name[1] 
    
    # En caso de encontrar duplicados:
    if duplicateFound:
        
        # Genera un numero aleatorio del 1 al 100
        num = random.randint(0,100)
        
        # Añade el numero al correo
        new_name = new_name + str(num)
        
    # Regresa la direccion de correo
    return new_name + dom

Ahora, para aprovechar adecuadamente el propósito de esta función, creé un ciclo que volvería a ejecutar la función cuando fuera necesario mientras iteraba a través del atributo "Nombre". El ciclo seguiría volviendo a ejecutar la función hasta que se creara un nombre de correo electrónico único.

In [ ]:
emails = []

for name in df['NombreP']:
    
    # Llamando a la función del correo
    email = emailGen(name)
    
    # Generando un bucle hasta que sea unico
    while email in emails:
        
        # Añadiendo un numero al correo para evitar duplicados
        email = emailGen(name, duplicateFound=True)
    
    # Añadiendo los cambios
    emails.append(email)
    
df['CorreoP'] = emails

Después de generar todos los correos electrónicos, los asigné al atributo "Correo electrónico" del marco de datos. También puede hacer una verificación opcional para ver que cada correo electrónico sea único con el mismo método que las ID.

**Saving the Dataset**

Ahora que los datos están completos y si estaba codificando, siéntase libre de ver el marco de datos antes de decidir guardarlo. Si todo se ve bien, guarde el marco de datos como un archivo .csv con este simple comando:

In [ ]:
df.to_csv('dataset_Personas.csv')